In [1]:
pip install array2gif

In [2]:
!pip install pyunpack
!pip install patool

     |████████████████████████████████| 81kB 2.6MB/s 


In [3]:
import gzip
import shutil
import os
import nibabel as nib
from tifffile import imsave, imread
import numpy as np
import matplotlib.pyplot as plt
import imageio
from array2gif import write_gif
from PIL import Image
from torchvision import transforms, models
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms.functional as TF
from torch.utils.data import DataLoader, Dataset, random_split
from sklearn.model_selection import train_test_split
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.tensorboard import SummaryWriter
import random
import shutil
%matplotlib inline


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [5]:
# ONLY RUN THIS CELL IF THE DATA IS NOT AVAILABLE IN YOUR GOOGLE DRIVE
#!wget https://www.cbica.upenn.edu/MICCAI_BraTS2020_TrainingData -P '/content/drive/My Drive'

In [9]:
from pyunpack import Archive
Archive('/content/drive/My Drive/MICCAI_BraTS2020_TrainingData').extractall('/content')

In [10]:
# delete unnecessary files
os.remove('/content/MICCAI_BraTS2020_TrainingData/name_mapping.csv')
os.remove('/content/MICCAI_BraTS2020_TrainingData/survival_info.csv')

In [11]:
# First create the dataset for the extracted file
!python create_dataset.py

['BraTS20_Training_246_flair.nii.gz', 'BraTS20_Training_246_seg.nii.gz', 'BraTS20_Training_246_t1.nii.gz', 'BraTS20_Training_246_t1ce.nii.gz', 'BraTS20_Training_246_t2.nii.gz']
['BraTS20_Training_278_flair.nii.gz', 'BraTS20_Training_278_seg.nii.gz', 'BraTS20_Training_278_t1.nii.gz', 'BraTS20_Training_278_t1ce.nii.gz', 'BraTS20_Training_278_t2.nii.gz']
['BraTS20_Training_352_flair.nii.gz', 'BraTS20_Training_352_seg.nii.gz', 'BraTS20_Training_352_t1.nii.gz', 'BraTS20_Training_352_t1ce.nii.gz', 'BraTS20_Training_352_t2.nii.gz']
['BraTS20_Training_174_flair.nii.gz', 'BraTS20_Training_174_seg.nii.gz', 'BraTS20_Training_174_t1.nii.gz', 'BraTS20_Training_174_t1ce.nii.gz', 'BraTS20_Training_174_t2.nii.gz']
['BraTS20_Training_162_flair.nii.gz', 'BraTS20_Training_162_seg.nii.gz', 'BraTS20_Training_162_t1.nii.gz', 'BraTS20_Training_162_t1ce.nii.gz', 'BraTS20_Training_162_t2.nii.gz']
['BraTS20_Training_236_flair.nii.gz', 'BraTS20_Training_236_seg.nii.gz', 'BraTS20_Training_236_t1.nii.gz', 'BraTS20

In [12]:
# Remove '/content/MICCAI_BraTS2020_TrainingData' from disk to save space
shutil.rmtree('/content/MICCAI_BraTS2020_TrainingData')

In [13]:
# I have placed a manual seed in line 34 to make code reproducible

def get_data(flair_imgs_dir, t1ce_imgs_dir, t2_imgs_dir, gt_imgs_dir, train_val_test_split = (0.7, 0.15, 0.15)):
    
    # First we get the file paths for all the input images and the corresponding ground truth images
    flair_imgs_list = os.listdir(flair_imgs_dir)
    t1ce_imgs_list = os.listdir(t1ce_imgs_dir)
    t2_imgs_list = os.listdir(t2_imgs_dir)
    gt_imgs_list = os.listdir(gt_imgs_dir)
    
    flair_imgs_list.sort()
    t1ce_imgs_list.sort()
    t2_imgs_list.sort()
    gt_imgs_list.sort()
      
    # We want the image lists to contain file paths (not file names)
    for i in range(len(flair_imgs_list)):
        flair_imgs_list[i] = os.path.join(flair_imgs_dir, flair_imgs_list[i])
        t1ce_imgs_list[i] = os.path.join(t1ce_imgs_dir, t1ce_imgs_list[i])
        t2_imgs_list[i] = os.path.join(t2_imgs_dir, t2_imgs_list[i])
        gt_imgs_list[i] = os.path.join(gt_imgs_dir, gt_imgs_list[i])
    
    # Check if number of images in all lists are the same
    flag = len(np.unique(np.array([len(flair_imgs_list), len(t1ce_imgs_list), len(t2_imgs_list), len(gt_imgs_list)])))
    assert flag ==1, "The number of images in different inputted folders vary!"
    
    # Computing the length of the train validation and test subsets
    trn_subset_len = int((train_val_test_split[0])* len(flair_imgs_list))
    vld_subset_len = int((len(flair_imgs_list) - trn_subset_len)*((train_val_test_split[1])/(train_val_test_split[1]+train_val_test_split[2])))
    tst_subset_len = len(flair_imgs_list) - trn_subset_len - vld_subset_len
    
    
    # Shuffle the lists (all dir lists get shuffled the same way)
    np.random.seed(42)
    p = np.random.permutation(len(flair_imgs_list))
    flair_imgs_list = ((np.array(flair_imgs_list))[p]).tolist()
    t1ce_imgs_list = ((np.array(t1ce_imgs_list))[p]).tolist()
    t2_imgs_list = ((np.array(t2_imgs_list))[p]).tolist()
    gt_imgs_list = ((np.array(gt_imgs_list))[p]).tolist()
    
    
    # Perform train, validation and test split on data
    flair_trn = flair_imgs_list[:trn_subset_len]
    flair_vld = flair_imgs_list[trn_subset_len: trn_subset_len+vld_subset_len]
    flair_tst = flair_imgs_list[-tst_subset_len:]
    
    t1ce_trn = t1ce_imgs_list[:trn_subset_len]
    t1ce_vld = t1ce_imgs_list[trn_subset_len: trn_subset_len+vld_subset_len]
    t1ce_tst = t1ce_imgs_list[-tst_subset_len:]
    
    t2_trn = t2_imgs_list[:trn_subset_len]
    t2_vld = t2_imgs_list[trn_subset_len: trn_subset_len+vld_subset_len]
    t2_tst = t2_imgs_list[-tst_subset_len:]
    
    gt_trn = gt_imgs_list[:trn_subset_len]
    gt_vld = gt_imgs_list[trn_subset_len: trn_subset_len+vld_subset_len]
    gt_tst = gt_imgs_list[-tst_subset_len:]
       
    return (flair_trn, t1ce_trn, t2_trn, gt_trn), (flair_vld, t1ce_vld, t2_vld, gt_vld), (flair_tst, t1ce_tst, t2_tst, gt_tst)

In [14]:
# I have placed a manual seed in line 30 to make results reproducible

class Train_Dataset(Dataset):
    '''
    Custom dataset defined for our BraTs Data for training
    The data has been randomly cropped to 160*160
    '''
    
    def __init__(self, flair_imgs_list, t1ce_imgs_list, t2_imgs_list, gt_imgs_list):
        '''
        flair_imgs_list: a list containing all directories for "flair" modality images in the train data
        t1ce_imgs_list: a list containing all directories for "t1ce" modality images in the train data
        t2_imgs_list: a list containing all directories for "t2" modality images in the train data
        gt_imgs_list: a list containing all directories for "seg" modality images in the train data  -> segmented images
         
        '''
        self.flair_imgs_list = flair_imgs_list
        self.t1ce_imgs_list = t1ce_imgs_list
        self.t2_imgs_list = t2_imgs_list
        self.gt_imgs_list = gt_imgs_list
        
    def transform(self, flair_PIL, t1ce_PIL, t2_PIL, gt_PIL):
        '''
        Performs identical random transformation on flair, t1ce, t2 and gt images 
        
        Returns:
        (input_img, gt_img) --> all sizes: (160*160)
        '''
        # Get random crop coordinates
        torch.manual_seed(42)
        top, left, height, width = transforms.RandomCrop.get_params(flair_PIL, (160, 160))
        # Pass the coordinates to F.crop --> Results in 160*160 numpy arrays
        flair_np = np.array(TF.crop(flair_PIL, top, left, height, width))
        t1ce_np = np.array(TF.crop(t1ce_PIL, top, left, height, width))
        t2_np = np.array(TF.crop(t2_PIL, top, left, height, width))
        gt_np = np.array(TF.crop(gt_PIL, top, left, height, width))
        
        # Create ground_truth mask
        gt_3d = np.zeros((3,160,160))
        gt_3d[0,:,:] = (gt_np==1)
        gt_3d[1,:,:] = (gt_np==2)
        gt_3d[2,:,:] = (gt_np==4)
        
        # Stack the input images
        input_img = np.stack((flair_np, t1ce_np, t2_np), axis=0)
        
        # Return input as a C*H*W tensor and ground_truth mask as  3*H*W tensor -> 3 is for the 3 types of tumors
        return (torch.tensor(input_img, dtype=torch.float)), (torch.tensor(gt_3d, dtype=torch.float))
   
        
    def __len__(self):
        '''returns the length of the entire dataset'''
        return len(self.flair_imgs_list)
        
    def __getitem__(self, idx):
        # Open Images
        flair_PIL = Image.open(self.flair_imgs_list[idx])
        t1ce_PIL = Image.open(self.t1ce_imgs_list[idx])
        t2_PIL = Image.open(self.t2_imgs_list[idx])
        gt_PIL = Image.open(self.gt_imgs_list[idx])
        
        # Perform Transformation
        input_img, gt_3d = self.transform(flair_PIL= flair_PIL, t1ce_PIL= t1ce_PIL, t2_PIL= t2_PIL, gt_PIL= gt_PIL)
                
        return input_img, gt_3d

In [15]:
class vld_tst_Dataset(Dataset):
    '''
    Custom dataset defined for our BraTs Data for validation and testing
    The data has been center cropped to 160*160
    '''
    
    def __init__(self, flair_imgs_list, t1ce_imgs_list, t2_imgs_list, gt_imgs_list):
        '''
        flair_imgs_list: a list containing all directories for "flair" modality images in the validation and test data
        t1ce_imgs_list: a list containing all directories for "t1ce" modality images in the validation and test data
        t2_imgs_list: a list containing all directories for "t2" modality images in the validation and test data
        gt_imgs_list: a list containing all directories for "seg" modality images in the validation and test data  -> segmented images
         
        '''
        self.flair_imgs_list = flair_imgs_list
        self.t1ce_imgs_list = t1ce_imgs_list
        self.t2_imgs_list = t2_imgs_list
        self.gt_imgs_list = gt_imgs_list
        self.center_crop= transforms.Compose([
            transforms.CenterCrop(160)         
        ])
        
        
    def transform(self, flair_PIL, t1ce_PIL, t2_PIL, gt_PIL):
        '''
        Performs required transformations on images
        '''     
        flair_np = np.array(self.center_crop(flair_PIL))
        t1ce_np = np.array(self.center_crop(t1ce_PIL))
        t2_np = np.array(self.center_crop(t2_PIL))
        gt_np = np.array(self.center_crop(gt_PIL))
        
        # Create ground_truth mask
        gt_3d = np.zeros((3,160,160))
        gt_3d[0,:,:] = (gt_np==1)
        gt_3d[1,:,:] = (gt_np==2)
        gt_3d[2,:,:] = (gt_np==4)
              
        # Stack input images together
        input_img = np.stack((flair_np, t1ce_np, t2_np), axis=0)
        
        # Return input as a C*H*W tensor and ground_truth mask as  3*H*W tensor --> 3 is for the 3 types of tumors
        return torch.tensor(input_img, dtype=torch.float), torch.tensor(gt_3d, dtype=torch.float)
    
        
    def __len__(self):
        '''returns the length of the entire dataset'''
        return len(self.flair_imgs_list)
        
    def __getitem__(self, idx):
        # Open Images
        flair_PIL = Image.open(self.flair_imgs_list[idx])
        t1ce_PIL = Image.open(self.t1ce_imgs_list[idx])
        t2_PIL = Image.open(self.t2_imgs_list[idx])
        gt_PIL = Image.open(self.gt_imgs_list[idx])
        
        # Perform Transformation
        input_img, gt_3d = self.transform(flair_PIL= flair_PIL, t1ce_PIL= t1ce_PIL, t2_PIL= t2_PIL, gt_PIL= gt_PIL)
                
        return input_img, gt_3d

In [16]:
flair_imgs_dir = './dataset/flair'
t1ce_imgs_dir = './dataset/t1ce'
t2_imgs_dir = './dataset/t2'
gt_imgs_dir = './dataset/segmentation'

In [17]:
# Performing the get_data
train_tuple, validation_tuple, test_tuple = get_data(flair_imgs_dir=flair_imgs_dir, t1ce_imgs_dir=t1ce_imgs_dir, t2_imgs_dir=t2_imgs_dir, gt_imgs_dir=gt_imgs_dir)

In [18]:
torch.manual_seed(42)
# Instanciating train, validation and test datasets
train_dataset = Train_Dataset(train_tuple[0], train_tuple[1], train_tuple[2], train_tuple[3])
val_dataset = vld_tst_Dataset(validation_tuple[0], validation_tuple[1], validation_tuple[2], validation_tuple[3])
test_dataset = vld_tst_Dataset(test_tuple[0], test_tuple[1], test_tuple[2], test_tuple[3])

In [19]:
torch.manual_seed(100)
# Define the Dataloader for each dataset
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, pin_memory=True)
validation_loader = DataLoader(val_dataset, batch_size=64, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=64, pin_memory=True)


#  Defining the model and instantiating it

In [20]:
LEAKAGE = 0.01

In [21]:
# Defining the convolution block in Encoder
class ConvBlock(nn.Module):
    def __init__(self, in_dim, out_dim, kernel, stride, padding=1):
        super(ConvBlock, self).__init__()
        self.conv = nn.Conv2d(in_dim, out_dim, (kernel, kernel), stride=stride, padding=padding)
        self.batchnorm = nn.BatchNorm2d(out_dim)
    
    def forward(self, x):
        residual = self.conv(x)
        residual = self.batchnorm(residual)
        residual = F.leaky_relu(residual, LEAKAGE)
        return residual


In [22]:
# Defining the Deconvolution block in Decoder
class DeconvBlock(nn.Module):
    def __init__(self, in_dim, out_dim, kernel, stride, padding=1, scale_factor=2):
        super(DeconvBlock, self).__init__()
        self.upsample = nn.Upsample(scale_factor=scale_factor, mode = 'bilinear')
        self.conv = nn.Conv2d(in_dim, out_dim, (kernel, kernel), stride=stride, padding=padding)
        self.batchnorm = nn.BatchNorm2d(out_dim)
    
    def forward(self, x):
        residual = self.upsample(x)
        residual = self.conv(residual)
        residual = self.batchnorm(residual)
        residual = F.relu(residual)
        return residual

In [23]:
# Defining the Segmentor net
class Segmentor(nn.Module):
    def __init__(self, d_in, d_out):
        super(Segmentor, self).__init__()
        
        # Encoder
        self.conv_1 = nn.Conv2d(d_in, 64, kernel_size=4, stride=2, padding=1)
        self.conv_2 = ConvBlock(64, 128, 4, 2, 1)
        self.conv_3 = ConvBlock(128, 256, 4, 2, 1)
        self.conv_4 = ConvBlock(256, 512, 4, 2, 1)
        
        # Decoder
        self.deconv_1 = DeconvBlock(512, 256, 3, 1, padding=1, scale_factor=2)
        self.deconv_2 = DeconvBlock(256, 128, 3, 1, padding=1, scale_factor=2)
        self.deconv_3 = DeconvBlock(128, 64, 3, 1, padding=1, scale_factor=2)
        self.deconv_4 = nn.Sequential(
            nn.Upsample(scale_factor=2, mode = 'bilinear'),
            nn.Conv2d(64, d_out, 3, 1, padding=1)
        )
        
    
    def forward(self, x):
        res_1 = self.conv_1(x)
        res_1 = F.leaky_relu(res_1)
        res_2 = self.conv_2(res_1)
        res_3 = self.conv_3(res_2)
        res_4 = self.conv_4(res_3)
        res_5 = self.deconv_1(res_4)
        # The "add" connection between corresponding encoder and decoder part
        res_5 = res_5 + res_3
        res_6 = self.deconv_2(res_5)
        # The "add" connection between corresponding encoder and decoder part
        res_6 = res_6 + res_2
        res_7 = self.deconv_3(res_6)
        # The "add" connection between corresponding encoder and decoder part
        res_7 = res_7 + res_1
        res_8 = self.deconv_4(res_7)
        
        return res_8
        

In [24]:
class Critic(nn.Module):
    def __init__(self, d_in):
        '''
        d_in: number of channels in the first convolutional layer.
        '''
        super(Critic, self).__init__()
        self.conv_1 = nn.Conv2d(d_in, 64, kernel_size=4, stride=2, padding=1)
        self.conv_2 = ConvBlock(64, 128, 4, 2, 1)
        self.conv_3 = ConvBlock(128, 256, 4, 2, 1)
         
    def forward(self, predicted_seg, gt_seg, input_img):
        gt_masked_imgs = gt_seg.unsqueeze(1) * input_img
        predicted_masked_imgs = predicted_seg.unsqueeze(1) * input_img
        
        # pass the predicted_masked_imgs through the net --> requires gradient
        res_1 = predicted_masked_imgs
        res_2 = self.conv_1(res_1)
        res_3 = self.conv_2(res_2)
        res_4 = self.conv_3(res_3)
        
        # Reshaping the resulted images for concatenation
        res_1 = res_1.reshape((64,-1))
        res_2 = res_2.reshape((64,-1))
        res_3 = res_3.reshape((64,-1))
        res_4 = res_4.reshape((64,-1))
        # Perform concatenation
        res_concat = torch.cat((res_1, res_2, res_3, res_4), dim=1)

        # Pass ground truth masked images forward through the network
        res_gt_1 = gt_masked_imgs
        res_gt_2 = self.conv_1(res_gt_1)
        res_gt_3 = self.conv_2(res_gt_2)
        res_gt_4 = self.conv_3(res_gt_3)
        
        res_gt_1 = res_gt_1.reshape((64,-1))
        res_gt_2 = res_gt_2.reshape((64,-1))
        res_gt_3 = res_gt_3.reshape((64,-1))
        res_gt_4 = res_gt_4.reshape((64,-1))
        # Perform concatenation
        res_gt_concat = torch.cat((res_gt_1, res_gt_2, res_gt_3, res_gt_4), dim=1)
    

        return res_concat - res_gt_concat

#### Creating Evaluation Metrics

In [25]:
def multi_scale_L1_loss(c_output, number_of_critic_scales=4):
  '''
  The critic_output is the difference between the concatenated vectors(Multi-scale) of the generated segmentation passed through the critic and the ground truth segmentation passed through it.
  the result will be a vector of size: (Batch_size*_)
  '''

  c_output = torch.abs(c_output)
  c_output = torch.sum(c_output, dim=1, keepdim=True)
  c_output = (1/number_of_critic_scales) * c_output
  c_output = torch.mean(c_output)

  return c_output

In [26]:
def dice_loss(s_output, s_target):
  '''
  s_output: Segmentor's output
  s_target: Segmentor's target --> the desired segmentation
  '''

  num = s_output * s_target
  den1 = s_output * s_output
  den2 = s_target * s_target

  dice = (2 * torch.sum(num))/(torch.sum(den1) + torch.sum(den2))
  
  # divide by batch size
  dice = 1/(s_output.size()[0]) * dice

  return 1 - dice

In [27]:
def custom_replace(tensor, on_zero, on_non_zero):
    # we create a copy of the original tensor, 
    # because of the way we are replacing them.
    res = tensor.clone()
    res[tensor==0] = on_zero
    res[tensor!=0] = on_non_zero
    return res

In [28]:
def segmentor_evaluation(s_output, s_target):
  """
  This function creates the batch_averaged FP(flase positive), TP(true positive), FN(flase negative) and TN(true negative) values for the segmented output
  THE ELEMENTS OF THE INPUTS ARE ASSUMED TO BE 0 OR 1.
  """
  batch_size = s_output.size()[0]
  s_output_inversed = custom_replace(s_output, 1, 0)
  s_target_inversed = custom_replace(s_target, 1, 0)
  # The true_positive value where s_output and s_target are both 1
  TP = (1)/(batch_size) *(torch.sum(s_output * s_target))

  # The false_positive value  is where s_output=1 and s_target=0(i.e.  s_target_inversed=1)
  FP = (1)/(batch_size) *torch.sum(s_output * s_target_inversed)

  # The true_negative value is where s_output=0 (i.e. s_output_inversed=1) and s_target=0 (i.e. s_target_inversed=1)
  TN = (1)/(batch_size) *torch.sum(s_output_inversed * s_target_inversed)

  # The false_negative is where s_output=0 (i.e. s_output_inversed=1) and s_target=1
  FN = (1)/(batch_size) *torch.sum(s_output_inversed * s_target)

  return TP, FP, TN, FN

In [29]:
def recall(TP, FN):
  '''
  Computes the recall value of the model
  '''
  recall = (TP)/(TP + FN)
  return recall

In [30]:
def precision(TP, FP):
  '''
  Computes the precision value of the model
  '''
  precision = (TP)/(TP + FP)
  return precision

In [31]:
def false_positive_rate(FP, TN):
  '''
  Computes the false positive rate of the model
  '''
  fpr = (FP)/(FP + TN)
  return fpr

In [32]:
def accuracy(TP, FP, TN, FN):
  '''
  Computes the accuracy of the model
  '''
  accuracy = (TP+TN)/(TP+FP+TN+FN)
  return accuracy

#### Instantiating the model and training it

In [33]:
# We try to implement the S3-3C network in this project which consists of 1 segmentor generating a 3channel output and 3 critics (one for every labels)
# The backpropagated loss will be the average of the 3 C's

In [34]:
torch.manual_seed(100)
# Instantiate the Segmentor(3 output channels) and 3 Critics (1 input channel)
segmentor = Segmentor(3,3)
critic_1 = Critic(3)
critic_2 = Critic(3)
critic_3 = Critic(3)

if torch.cuda.is_available():
  segmentor = segmentor.cuda()
  critic_1 = critic_1.cuda()
  critic_2 = critic_2.cuda()
  critic_3 = critic_3.cuda()

In [35]:
learning_rate = 0.00002

In [36]:
torch.manual_seed(42)
# Define the optimizers
s_parameters = segmentor.parameters()
optimizer_s = torch.optim.RMSprop(s_parameters, lr=learning_rate)

c_parameters = list(critic_1.parameters()) + list(critic_2.parameters()) + list(critic_3.parameters())
optimizer_c = torch.optim.RMSprop(c_parameters, lr=learning_rate)

In [37]:
# Define the number of epochs
epochs = 10

In [38]:
#shutil.rmtree('runs')

In [39]:
# Define a binarization threshold (for determining segmentor decision in evaluation phase):
threshold = 0.5

In [40]:
# Create a directory to log values
!mkdir ./runs

In [41]:
# Instantiate a Tensorboard summary writer 
writer = SummaryWriter(log_dir= "./runs")

In [42]:
###################
# Training process
###################

for i in range(epochs):
  # Set the model to train mode
  segmentor.train()
  critic_1.train()
  critic_2.train()
  critic_3.train()

  for j, (input_img, gt_3d) in enumerate(train_loader):
    
    # Transfer loaded data to GPU if cuda is available
    if torch.cuda.is_available:
      input_img = input_img.cuda()
      gt_3d = gt_3d.cuda()

    # "TRAIN C NET"
    optimizer_c.zero_grad()
    # Pass input images forward through the segmentor
    output_s = segmentor.forward(input_img)
    
    # For Critic_1
    output_1 = output_s[:,0,:,:]
    target_1 = gt_3d[:,0,:,:]
    # For Critic_2
    output_2 = output_s[:,1,:,:]
    target_2 = gt_3d[:,1,:,:]
    # For Critic_3
    output_3 = output_s[:,2,:,:]
    target_3 = gt_3d[:,2,:,:]

        

    # Forward pass through critic nets
    output_c1 = critic_1.forward(predicted_seg= output_1.detach(), gt_seg= target_1.detach(), input_img = input_img.clone())
    output_c2 = critic_2.forward(predicted_seg= output_2.detach(), gt_seg= target_2.detach(), input_img = input_img.clone())
    output_c3 = critic_3.forward(predicted_seg= output_3.detach(), gt_seg= target_3.detach(), input_img = input_img.clone())
    
    # Compute loss due to each critic net
    loss1 = multi_scale_L1_loss(c_output= output_c1)
    loss2 = multi_scale_L1_loss(c_output= output_c2)
    loss3 = multi_scale_L1_loss(c_output= output_c3)
    
    # Compute average Loss
    # Multiply by (-1) because we want the critic to MAXIMIZE the loss during training
    loss_c = (-1)*(loss1 + loss2 + loss3)/3
    
    # Backprob & update critic parameters
    loss_c.backward()
    optimizer_c.step()



    # "TRAIN S NET"
    optimizer_s.zero_grad()
    # The same input_img is required to be passed through the same net --> forward pass is not necessary and we can use output_s from before
    
    # For Critic_1
    output_1 = output_s[:,0,:,:]
    target_1 = gt_3d[:,0,:,:]
    # For Critic_2
    output_2 = output_s[:,1,:,:]
    target_2 = gt_3d[:,1,:,:]
    # For Critic_3
    output_3 = output_s[:,2,:,:]
    target_3 = gt_3d[:,2,:,:]

    # Forward pass through critic nets --> this time we should not use detach
    output_c1 = critic_1.forward(predicted_seg= output_1, gt_seg= target_1, input_img = input_img.clone())
    output_c2 = critic_2.forward(predicted_seg= output_2, gt_seg= target_2, input_img = input_img.clone())
    output_c3 = critic_3.forward(predicted_seg= output_3, gt_seg= target_3, input_img = input_img.clone())
    
    # Compute loss due to each critic net
    loss1 = multi_scale_L1_loss(c_output= output_c1)
    loss2 = multi_scale_L1_loss(c_output= output_c2)
    loss3 = multi_scale_L1_loss(c_output= output_c3)

    # Compute dice loss
    loss_dice = dice_loss(output_s, gt_3d)

    # Compute average Loss
    loss_s = (loss1 + loss2 + loss3)/3 + loss_dice

    # Backprob & update segmentor parameters
    loss_s.backward()
    optimizer_s.step()

    # Compute evaluation metrics and save values to tensorboard in the second to last epoch
    if  j== len(train_loader)-2 :
      # Compute metric values
      
      TP_trn, FP_trn, TN_trn, FN_trn = segmentor_evaluation(s_output= output_s, s_target= gt_3d)
      recall_trn = recall(TP= TP_trn, FN= FN_trn)
      precision_trn = precision(TP= TP_trn, FP= FP_trn)
      false_positive_rate_trn = false_positive_rate(FP=FP_trn, TN=TN_trn)
      accuracy_trn = accuracy(TP=TP_trn, FP=FP_trn, TN=TN_trn, FN=FN_trn)

      # Save computed metrics and losses
      writer.add_scalar('Recall/train', recall_trn, i+1)
      writer.add_scalar('Precision/train', precision_trn, i+1)
      writer.add_scalar('False Positive Rate/train', false_positive_rate_trn, i+1)
      writer.add_scalar('Accuracy/train', accuracy_trn, i+1)
      writer.add_scalar('Dice Loss/train', loss_dice, i+1)
      writer.add_scalar('Critic Loss/train', loss_c, i+1)
      writer.add_scalar('Segmentor Loss/train', loss_s, i+1)

####################
# Evaluation process
####################

  # After each epoch we evaluate the model and save evaluation metrics
  with torch.no_grad():
    # Set models on evaluation mode
    segmentor.eval()
    critic_1.eval()
    critic_2.eval()
    critic_3.eval()

    recall_val_avg = 0
    precision_val_avg = 0
    false_positive_rate_val_avg = 0
    accuracy_val_avg = 0
    loss_dice_val_avg = 0
    loss_c_val_avg = 0
    loss_s_val_avg = 0

    for j, (input_img_val, gt_3d_val) in enumerate(validation_loader):
      # Transfer loaded data to GPU if cuda is available
      if torch.cuda.is_available:
        input_img_val = input_img_val.cuda()
        gt_3d_val = gt_3d_val.cuda()
     
      
      # Pass input_img forward through the segmentor
      output_s_val = segmentor.forward(input_img_val)
      
      # We binarize the output of the segmentor to determine it's decision for every pixel of the input images
      output_s_val[output_s_val >= threshold] = 1.0
      output_s_val[output_s_val < threshold]  = 0.0

      # Now the output of the segmentor can be used to compute the recall, precision, false_positive_rate and accuracy of the segmentor
      TP_val, FP_val, TN_val, FN_val = segmentor_evaluation(s_output= output_s_val, s_target= gt_3d_val)
      recall_val = recall(TP = TP_val, FN= FN_val)
      precision_val = precision(TP= TP_val, FP= FP_val)
      false_positive_rate_val = false_positive_rate(FP= FP_val, TN= TN_val)
      accuracy_val = accuracy(TP= TP_val, FP= FP_val, TN= TN_val, FN= FN_val)

      # For Critic_1
      output_1_val = output_s_val[:,0,:,:]
      target_1_val = gt_3d_val[:,0,:,:]
      # For Critic_2
      output_2_val = output_s_val[:,1,:,:]
      target_2_val = gt_3d_val[:,1,:,:]
      # For Critic_3
      output_3_val = output_s_val[:,2,:,:]
      target_3_val = gt_3d_val[:,2,:,:]


      # Forward pass through critic nets --> this time we should not use detach
      output_c1_val = critic_1.forward(predicted_seg= output_1_val, gt_seg= target_1_val, input_img = input_img_val.clone())
      output_c2_val = critic_2.forward(predicted_seg= output_2_val, gt_seg= target_2_val, input_img = input_img_val.clone())
      output_c3_val = critic_3.forward(predicted_seg= output_3_val, gt_seg= target_3_val, input_img = input_img_val.clone())

      # Compute the loss_s, loss_c and loss_dice for the validation data
      loss_dice_val = dice_loss(output_s_val, gt_3d_val)

      # Compute loss due to each critic net
      loss1_val = multi_scale_L1_loss(c_output= output_c1_val)
      loss2_val = multi_scale_L1_loss(c_output= output_c2_val)
      loss3_val = multi_scale_L1_loss(c_output= output_c3_val)
      # Compute the loss_s, loss_c for the validation data
      loss_c_val = (-1)*(loss1_val + loss2_val + loss3_val)/3
      loss_s_val = (loss1_val + loss2_val + loss3_val)/3 + loss_dice_val

      # Save the Computed parameters --> to be averaged
      recall_val_avg += recall_val
      precision_val_avg += precision_val
      false_positive_rate_val_avg += false_positive_rate_val
      accuracy_val_avg += accuracy_val
      loss_dice_val_avg += loss_dice_val
      loss_c_val_avg += loss_c_val
      loss_s_val_avg += loss_s_val

    # After all the validation data has been passed through the network, compute the average evaluation metrics
    recall_val_avg = recall_val_avg / len(validation_loader)
    precision_val_avg = precision_val_avg / len(validation_loader)
    false_positive_rate_val_avg = false_positive_rate_val_avg / len(validation_loader)
    accuracy_val_avg = accuracy_val_avg / len(validation_loader)
    loss_dice_val_avg = loss_dice_val_avg / len(validation_loader)
    loss_c_val_avg = loss_c_val_avg / len(validation_loader)
    loss_s_val_avg = loss_s_val_avg / len(validation_loader)

    # Save computed losses and metric values via tensorboard
    writer.add_scalar('Recall/validation', recall_val_avg, i+1)
    writer.add_scalar('Precision/validation', precision_val_avg, i+1)
    writer.add_scalar('False Positive Rate/validation', false_positive_rate_val_avg, i+1)
    writer.add_scalar('Accuracy/validation', accuracy_val_avg, i+1)
    writer.add_scalar('Dice Loss/validation', loss_dice_val_avg, i+1)
    writer.add_scalar('Critic Loss/validation', loss_c_val_avg, i+1)
    writer.add_scalar('Segmentor Loss/validation', loss_s_val_avg, i+1)

  # Print results at the end of earch epoch
  print(f"==> EPOCH {i+1}({i+1}/{epochs}) Train Dice Loss: {loss_dice:.8f}   Validation Dice Loss: {loss_dice_val:.8f}")
  print(f"==> EPOCH {i+1}({i+1}/{epochs}) Segmentor Loss:{loss_s:.4f}")
  print(f"==> EPOCH {i+1}({i+1}/{epochs}) Critic Loss:{loss_c:.4f} \n")
  


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2973: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))


==> EPOCH 1(1/10) Train Dice Loss: 1.00000358   Validation Dice Loss: 0.99919385
==> EPOCH 1(1/10) Segmentor Loss:9610171.0000
==> EPOCH 1(1/10) Critic Loss:-9608317.0000 

==> EPOCH 2(2/10) Train Dice Loss: 1.00002944   Validation Dice Loss: 0.99915594
==> EPOCH 2(2/10) Segmentor Loss:7237510.0000
==> EPOCH 2(2/10) Critic Loss:-7236241.0000 

==> EPOCH 3(3/10) Train Dice Loss: 0.99994254   Validation Dice Loss: 0.99913794
==> EPOCH 3(3/10) Segmentor Loss:6702974.0000
==> EPOCH 3(3/10) Critic Loss:-6701923.0000 

==> EPOCH 4(4/10) Train Dice Loss: 1.00003660   Validation Dice Loss: 0.99913329
==> EPOCH 4(4/10) Segmentor Loss:6789569.0000
==> EPOCH 4(4/10) Critic Loss:-6788577.5000 

==> EPOCH 5(5/10) Train Dice Loss: 0.99989015   Validation Dice Loss: 0.99915493
==> EPOCH 5(5/10) Segmentor Loss:6776188.0000
==> EPOCH 5(5/10) Critic Loss:-6775256.0000 

==> EPOCH 6(6/10) Train Dice Loss: 1.00004518   Validation Dice Loss: 0.99913567
==> EPOCH 6(6/10) Segmentor Loss:6875981.0000
==> EPOC

In [43]:
# Save model state_dict
torch.save(segmentor.state_dict(), '/content/drive/My Drive/segmentor_1.pt')
torch.save(critic_1.state_dict(), '/content/drive/My Drive/critic1_1.pt')
torch.save(critic_2.state_dict(), '/content/drive/My Drive/critic2_1.pt')
torch.save(critic_3.state_dict(), '/content/drive/My Drive/critic3_1.pt')

In [45]:
# Run on test data
with torch.no_grad():
  #
  for j, (input_img_tst, gt_3d_tst) in enumerate(test_loader):
  # Transfer loaded data to GPU if cuda is available
    if torch.cuda.is_available:
      input_img_tst = input_img_tst.cuda()
      gt_3d_tst = gt_3d_tst.cuda()
    
    # Pass input_img forward through the segmentor
    output_s_tst = segmentor.forward(input_img_tst)
    
    # We binarize the output of the segmentor to determine it's decision for every pixel of the input images
    output_s_tst[output_s_tst >= threshold] = 1.0
    output_s_tst[output_s_tst < threshold]  = 0.0

    # Now the output of the segmentor can be used to compute the recall, precision, false_positive_rate and accuracy of the segmentor
    TP_tst, FP_tst, TN_tst, FN_tst = segmentor_evaluation(s_output= output_s_tst, s_target= gt_3d_tst)
    recall_tst = recall(TP = TP_tst, FN= FN_tst)
    precision_tst = precision(TP= TP_tst, FP= FP_tst)
    false_positive_rate_tst = false_positive_rate(FP= FP_tst, TN= TN_tst)
    accuracy_tst = accuracy(TP= TP_tst, FP= FP_tst, TN= TN_tst, FN= FN_tst)


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2973: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))


In [46]:
recall_tst

tensor(0.3774, device='cuda:0')

In [47]:
precision_tst

tensor(0.0235, device='cuda:0')

In [ ]:
len(validation_loader)

111

In [ ]:
len(val_dataset)

7085